In [14]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import SQLContext

In [13]:
spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

raw_events = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "planes") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40627)
Traceback (most recent call last):
  File "/spark-2.2.0-bin-hadoop2.6/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/spark-2.2.0-bin-hadoop2.6/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40627)

In [3]:
raw_events.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [4]:
raw_events.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
events = raw_events.select(raw_events.value.cast('string'))

In [6]:
sql_context = SQLContext(spark)
parsed_json = sql_context.read.json(events.rdd.map(lambda r: r.value))
parsed_json.printSchema()

root
 |-- states: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- time: long (nullable = true)



In [7]:
parsed_json.show()

+--------------------+----------+
|              states|      time|
+--------------------+----------+
|[WrappedArray(a8a...|1626045632|
|[WrappedArray(a8a...|1626045632|
|[WrappedArray(e49...|1626046480|
|[WrappedArray(a6b...|1626046830|
|                null|1626052790|
|                null|1626053410|
+--------------------+----------+



In [ ]:
parsed_json.withColumn(parsed_json['states'])

In [8]:
states_col = parsed_json['states']

In [11]:
parsed_json.withColumn("state0",states_col.getItem(0))

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40627)
Traceback (most recent call last):
  File "/spark-2.2.0-bin-hadoop2.6/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/spark-2.2.0-bin-hadoop2.6/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40627)